In [1]:
data_folder = '..\DATA'

golden_globe_years_filename = 'academy_award_years.json'
golden_globe_categories_filename = 'academy_award_categories.json'
film_info_filename = 'film_info.json'
person_info_filename = 'person_info.json'

wiki_base_url = 'https://en.wikipedia.org'
init_url = '/wiki/76th_Golden_Globe_Awards'

In [2]:
from datetime import datetime
def try_strptime(s, fmts=['%B %d, %Y','%Y-%m-%d', '%A, %B %d, %Y']):
    for fmt in fmts:
        try:
            return datetime.strptime(s, fmt).strftime("%Y-%m-%d")
        except:
            continue

    return None

In [6]:
from bs4 import BeautifulSoup
import requests
import json

award_years = []
categories = {}
persons = {}
with open(data_folder + '\\' + person_info_filename, 'r') as infile:
    persons = json.loads(infile.read())
films = {}
with open(data_folder + '\\' + film_info_filename, 'r') as infile:
    films = json.loads(infile.read())

def get_nomination_info(block, category_type, won=False):
    film_info = block
    for i_elem in block.find_all('i'):
        if i_elem.find('a'):
            film_info = i_elem
            break
    film_link = film_info.find('a')
    film_id = None
    if film_link is not None:
        film_id = film_link['href']
        film_title = film_link.text
        print('FILM--', film_title, 'won:', won)
        if film_id not in films:
            films[film_id] = {'title': film_title }
    
    nomination_persons = []
    person_names = []
    if 'is_actor' in category_type:
        link = block.find('a')
        person_id = link['href']
        person_name = link.text
        nomination_persons.append(person_id)
        person_names.append(person_name)
        if person_id not in persons:
            persons[person_id] = { 'name': person_name }
    else:
        for link in block.find_all('a'):
            if link['href'] == film_id:
                continue
            person_id = link['href']
            person_name = link.text
            nomination_persons.append(person_id)
            person_names.append(person_name)
            if person_id not in persons:
                persons[person_id] = { 'name': person_name }
    print("PERS---", ", ".join(person_names))
    return {
        'film_id': film_id,
        'persons': nomination_persons,
        'won': won
    }

def get_winner_info(nomination_info, category_type):
    winner_block = nomination_info.find('b')
    return get_nomination_info(winner_block, category_type, won=True)

def get_category_info(category):
    category_link = category.find('a')
    category_id = category_link['href']
    category_name = category_link.text
    if category_id not in categories:
        categories[category_id] = category_name
    print('CAT -', category_name)
    category_type = {}
    if 'actor' in category_name.lower() or 'actress' in category_name.lower():
        category_type['is_actor'] = True
    if 'adapted' in category_name.lower():
        category_type['is_adapted_screenplay'] = True
    nominations = []
    winner_list = category.find('ul')
    winner_info = winner_list.find('li')
    while winner_info is not None:
        if winner_info.find('b') is not None:
            nominations.append(get_winner_info(winner_info, category_type))
        else:
            nominations.append(get_nomination_info(winner_info, category_type))
        nomination_list = winner_info.find('ul')
        if nomination_list is not None:
            for nomination in nomination_list.find_all('li'):
                nominations.append(get_nomination_info(nomination, category_type))
        winner_info = winner_info.find_next_sibling('li')
    # film info
    return {
        'id': category_id,
        'nominations' : nominations
    }

def parse_date(info_box):
    date_row_header = info_box.find('th', string='Date')
    if date_row_header is not None:
        date_row = date_row_header.find_next_sibling()
        full_text = date_row.text
        span_elem = date_row.find('span')
        if span_elem is not None:
            full_text = full_text.replace(span_elem.text, '')
        small_elem = date_row.find('small')
        if small_elem is not None:
            full_text = full_text.replace(small_elem.text, '')
        sup_elem = date_row.find('sup')
        if sup_elem is not None:
            full_text = full_text.replace(sup_elem.text, '')
        return try_strptime(full_text)
    
def get_headline(soup):
    for headline_id in ['Winners_and_nominees']: 
        awards_headline = soup.find(id=headline_id, class_='mw-headline')
        if awards_headline is not None:
            return awards_headline
    return None
    
def get_awards_table(soup):
    awards_headline = get_headline(soup)
    awards_header = awards_headline.find_parent('h3')
    if awards_header is None:
        awards_header = awards_headline.find_parent('h2')
    awards_table = awards_header.find_next_sibling('table', class_='wikitable')
    if awards_table is not None:
        return awards_table
    div_elem = awards_header.find_next_sibling('div')
    if div_elem is not None:
        return div_elem.find('table', class_='wikitable')
        

def get_golden_globes_page(awards_page_url):
    response = requests.get(wiki_base_url + awards_page_url)
    html = response.text

    soup = BeautifulSoup(html, "html.parser")
    
    info_box = soup.find(class_='infobox vevent')
    
    # get info for awards year
    award_year_info = {}
    
    award_year_info['name'] = info_box.find_all('tr')[0].text
    
    award_year_info['date'] = parse_date(info_box)
    
    print(award_year_info['name'], award_year_info['date'])
    
    # get categories
    award_year_info['categories'] = []

    awards_table = get_awards_table(soup)
    for double_category_header in awards_table.find_all('tr'):
        next_row = double_category_header.find_next_sibling('tr')
        if next_row is None:
            continue
        double_category_info = next_row.find('td')
        if double_category_info is None:
            continue
        first_category_header = double_category_header.find_all('a')[0]['title']
        second_category_header = double_category_header.find_all('a')[1]['title']
        
        
    award_years.append(award_year_info)
        
    # get previous year url
    last_row = info_box.find_all('tr')[-1]
    main_awards_url = last_row.find(text='Golden Globes')
    previous_li = main_awards_url.find_parent('li').find_previous_sibling('li')
    if previous_li is None:
        return None
    previous_li_link = previous_li.find('a')
    if previous_li_link is None:
        return None
    return previous_li_link['href']
    
next_url = get_golden_globes_page(init_url)
while next_url is not None:
    next_url = get_golden_globes_page(next_url)

76th Golden Globe Awards 2019-01-06
CAT - Bohemian Rhapsody
FILM-- Bohemian Rhapsody won: True
PERS--- 
FILM-- Black Panther won: False
PERS--- 
FILM-- BlacKkKlansman won: False
PERS--- 
FILM-- If Beale Street Could Talk won: False
PERS--- 
FILM-- A Star Is Born won: False
PERS--- 
CAT - Green Book
FILM-- Green Book won: True
PERS--- 
FILM-- Crazy Rich Asians won: False
PERS--- 
FILM-- The Favourite won: False
PERS--- 
FILM-- Mary Poppins Returns won: False
PERS--- 
FILM-- Vice won: False
PERS--- 
CAT - Rami Malek
FILM-- Bohemian Rhapsody won: True
PERS--- Rami Malek, Freddie Mercury
FILM-- A Star Is Born won: False
PERS--- Bradley Cooper
FILM-- At Eternity's Gate won: False
PERS--- Willem Dafoe, Vincent van Gogh
FILM-- Boy Erased won: False
PERS--- Lucas Hedges
FILM-- BlacKkKlansman won: False
PERS--- John David Washington, Ron Stallworth
CAT - Glenn Close
FILM-- The Wife won: True
PERS--- Glenn Close
FILM-- A Star Is Born won: False
PERS--- Lady Gaga
FILM-- Destroyer won: False
PERS-

74th Golden Globe Awards 2017-01-08
CAT - Moonlight
FILM-- Moonlight won: True
PERS--- 
FILM-- Hacksaw Ridge won: False
PERS--- 
FILM-- Hell or High Water won: False
PERS--- 
FILM-- Lion won: False
PERS--- 
FILM-- Manchester by the Sea won: False
PERS--- 
CAT - La La Land
FILM-- La La Land won: True
PERS--- 
FILM-- 20th Century Women won: False
PERS--- 
FILM-- Deadpool won: False
PERS--- 
FILM-- Florence Foster Jenkins won: False
PERS--- 
FILM-- Sing Street won: False
PERS--- 
CAT - Casey Affleck
FILM-- Manchester by the Sea won: True
PERS--- Casey Affleck
FILM-- Loving won: False
PERS--- Joel Edgerton
FILM-- Hacksaw Ridge won: False
PERS--- Andrew Garfield, Desmond T. Doss
FILM-- Captain Fantastic won: False
PERS--- Viggo Mortensen
FILM-- Fences won: False
PERS--- Denzel Washington
CAT - Isabelle Huppert
FILM-- Elle won: True
PERS--- Isabelle Huppert
FILM-- Arrival won: False
PERS--- Amy Adams
FILM-- Miss Sloane won: False
PERS--- Jessica Chastain
FILM-- Loving won: False
PERS--- Ruth

72nd Golden Globe Awards 2015-01-11
CAT - Boyhood
FILM-- Boyhood won: True
PERS--- 
FILM-- Foxcatcher won: False
PERS--- 
FILM-- The Imitation Game won: False
PERS--- 
FILM-- Selma won: False
PERS--- 
FILM-- The Theory of Everything won: False
PERS--- 
CAT - The Grand Budapest Hotel
FILM-- The Grand Budapest Hotel won: True
PERS--- 
FILM-- Birdman won: False
PERS--- 
FILM-- Into the Woods won: False
PERS--- 
FILM-- Pride won: False
PERS--- 
FILM-- St. Vincent won: False
PERS--- 
CAT - Eddie Redmayne
FILM-- The Theory of Everything won: True
PERS--- Eddie Redmayne, Stephen Hawking
FILM-- Foxcatcher won: False
PERS--- Steve Carell, John du Pont
FILM-- The Imitation Game won: False
PERS--- Benedict Cumberbatch, Alan Turing
FILM-- Nightcrawler won: False
PERS--- Jake Gyllenhaal
FILM-- Selma won: False
PERS--- David Oyelowo, Martin Luther King Jr.
CAT - Julianne Moore
FILM-- Still Alice won: True
PERS--- Julianne Moore
FILM-- Cake won: False
PERS--- Jennifer Aniston
FILM-- The Theory of Eve

70th Golden Globe Awards 2013-01-13
CAT - Argo
FILM-- Argo won: True
PERS--- 
FILM-- Django Unchained won: False
PERS--- 
FILM-- Life of Pi won: False
PERS--- 
FILM-- Lincoln won: False
PERS--- 
FILM-- Zero Dark Thirty won: False
PERS--- 
CAT - Les Misérables
FILM-- Les Misérables won: True
PERS--- 
FILM-- The Best Exotic Marigold Hotel won: False
PERS--- 
FILM-- Moonrise Kingdom won: False
PERS--- 
FILM-- Salmon Fishing in the Yemen won: False
PERS--- 
FILM-- Silver Linings Playbook won: False
PERS--- 
CAT - Daniel Day-Lewis
FILM-- Lincoln won: True
PERS--- Daniel Day-Lewis, Abraham Lincoln
FILM-- Arbitrage won: False
PERS--- Richard Gere
FILM-- The Sessions won: False
PERS--- John Hawkes, Mark O'Brien
FILM-- The Master won: False
PERS--- Joaquin Phoenix
FILM-- Flight won: False
PERS--- Denzel Washington
CAT - Jessica Chastain
FILM-- Zero Dark Thirty won: True
PERS--- Jessica Chastain
FILM-- Rust and Bone won: False
PERS--- Marion Cotillard
FILM-- Hitchcock won: False
PERS--- Helen Mi

68th Golden Globe Awards 2011-01-16
CAT - The Social Network
FILM-- The Social Network won: True
PERS--- 
FILM-- Black Swan won: False
PERS--- 
FILM-- The Fighter won: False
PERS--- 
FILM-- Inception won: False
PERS--- 
FILM-- The King's Speech won: False
PERS--- 
CAT - The Kids Are All Right
FILM-- The Kids Are All Right won: True
PERS--- 
FILM-- Alice in Wonderland won: False
PERS--- 
FILM-- Burlesque won: False
PERS--- 
FILM-- Red won: False
PERS--- 
FILM-- The Tourist won: False
PERS--- 
CAT - Colin Firth
FILM-- Colin Firth won: True
PERS--- King George VI
FILM-- Jesse Eisenberg won: False
PERS--- Mark Zuckerberg
FILM-- 127 Hours won: False
PERS--- James Franco, Aron Ralston
FILM-- Blue Valentine won: False
PERS--- Ryan Gosling
FILM-- Mark Wahlberg won: False
PERS--- Micky Ward
CAT - Natalie Portman
FILM-- Natalie Portman won: True
PERS--- 
FILM-- Frankie & Alice won: False
PERS--- Halle Berry
FILM-- Rabbit Hole won: False
PERS--- Nicole Kidman
FILM-- Winter's Bone won: False
PERS-

...

43rd Golden Globe Awards 1986-01-24
CAT - Out of Africa
FILM-- Out of Africa won: True
PERS--- 
FILM-- The Color Purple won: False
PERS--- 
FILM-- Kiss of the Spider Woman won: False
PERS--- 
FILM-- Runaway Train won: False
PERS--- 
FILM-- Witness won: False
PERS--- 
CAT - Prizzi's Honor
FILM-- Prizzi's Honor won: True
PERS--- 
FILM-- Back to the Future won: False
PERS--- 
FILM-- A Chorus Line won: False
PERS--- 
FILM-- Cocoon won: False
PERS--- 
FILM-- The Purple Rose of Cairo won: False
PERS--- 
CAT - Jon Voight
FILM-- Runaway Train won: True
PERS--- Jon Voight
FILM-- Witness won: False
PERS--- Harrison Ford
FILM-- Twice in a Lifetime won: False
PERS--- Gene Hackman
FILM-- Kiss of the Spider Woman won: False
PERS--- William Hurt
FILM-- Kiss of the Spider Woman won: False
PERS--- Raúl Juliá
CAT - Whoopi Goldberg
FILM-- The Color Purple won: True
PERS--- Whoopi Goldberg
FILM-- Agnes of God won: False
PERS--- Anne Bancroft
FILM-- Mask won: False
PERS--- Cher, Florence "Rusty" Dennis
FIL

41st Golden Globe Awards 1984-01-28
CAT - Terms of Endearment
FILM-- Terms of Endearment won: True
PERS--- 
FILM-- Reuben, Reuben won: False
PERS--- 
FILM-- The Right Stuff won: False
PERS--- 
FILM-- Silkwood won: False
PERS--- 
FILM-- Tender Mercies won: False
PERS--- 
CAT - Yentl
FILM-- Yentl won: True
PERS--- 
FILM-- The Big Chill won: False
PERS--- 
FILM-- Flashdance won: False
PERS--- 
FILM-- Trading Places won: False
PERS--- 
FILM-- Zelig won: False
PERS--- 
CAT - Tom Courtenay
FILM-- The Dresser won: True
PERS--- Tom Courtenay
FILM-- Tender Mercies won: True
PERS--- Robert Duvall
FILM-- Reuben, Reuben won: False
PERS--- Tom Conti
FILM-- The Grey Fox won: False
PERS--- Richard Farnsworth
FILM-- The Dresser won: False
PERS--- Albert Finney
FILM-- Scarface won: False
PERS--- Al Pacino, Tony Montana
FILM-- Star 80 won: False
PERS--- Eric Roberts, Paul Snider
CAT - Shirley MacLaine
FILM-- Terms of Endearment won: True
PERS--- Shirley MacLaine
FILM-- Testament won: False
PERS--- Jane 

39th Golden Globe Awards 1982-01-30
CAT - On Golden Pond
FILM-- On Golden Pond won: True
PERS--- 
FILM-- The French Lieutenant's Woman won: False
PERS--- 
FILM-- Prince of the City won: False
PERS--- 
FILM-- Ragtime won: False
PERS--- 
FILM-- Reds won: False
PERS--- 
CAT - Arthur
FILM-- Arthur won: True
PERS--- 
FILM-- The Four Seasons won: False
PERS--- 
FILM-- Pennies from Heaven won: False
PERS--- 
FILM-- S.O.B. won: False
PERS--- 
FILM-- Zoot Suit won: False
PERS--- 
CAT - Henry Fonda
FILM-- On Golden Pond won: True
PERS--- Henry Fonda
FILM-- Reds won: False
PERS--- Warren Beatty, John Reed
FILM-- Taps won: False
PERS--- Timothy Hutton
FILM-- Atlantic City won: False
PERS--- Burt Lancaster
FILM-- Prince of the City won: False
PERS--- Treat Williams
CAT - Meryl Streep
FILM-- The French Lieutenant's Woman won: True
PERS--- Meryl Streep
FILM-- Absence of Malice won: False
PERS--- Sally Field
FILM-- On Golden Pond won: False
PERS--- Katharine Hepburn
FILM-- Reds won: False
PERS--- Dian

37th Golden Globe Awards 1980-01-26


AttributeError: 'NoneType' object has no attribute 'find_parent'

In [4]:
print(len(persons.items()))
print(len(films.items()))

6974
4701


In [5]:
import os
from os.path import isfile

def save_as_json(filename, data):
    try:
        with open(data_folder + '\\' + filename, 'w') as outfile:
            outfile.write(json.dumps(data))
    except e as Exception:
        print("exception saving json,", e)
        
save_as_json(academy_award_years_filename, award_years)
save_as_json(academy_award_categories_filename, categories)
save_as_json(film_info_filename, films)
save_as_json(person_info_filename, persons)